In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-224190
Azure region: southcentralus
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-224190


In [2]:
import azureml.core
from azureml.exceptions import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "amlcomp"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target {}.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                               max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

print("Azure Machine Learning Compute attached")

Found existing compute target amlcomp.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Azure Machine Learning Compute attached


In [3]:
import azureml.core
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job

src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=16,
                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad
Web View: https://ml.azure.com/runs/HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-224190/workspaces/quick-starts-ws-224190&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-01T05:27:11.504561][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-01T05:27:12.6455540Z][SCHEDULER][INFO]Scheduling job, id='HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad_0' 
[2023-02-01T05:27:12.7855551Z][SCHEDULER][INFO]Scheduling job, id='HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad_1' 
[2023-02-01T05:27:12.8941651Z][SCHEDULER][INFO]Scheduling job, id='HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad_2' 
[2023-02-01T05:27:12.939979][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-02-01T05:27:13.0038790Z][SCHEDULER][INFO]Scheduling job, id='HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0

In [5]:
# pip install joblib
# pip install pip install azureml-sdk
# pip install scikit-learn
# pip install automl

In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_details()
hyperdrive_run.get_children_sorted_by_primary_metric(top=1)


# model = best_run.register_model(model_name='udacity-project', model_path='outputs/hd_bestrun_model.joblib')

[{'run_id': 'HD_64aeea36-8cdc-47eb-a2e2-6f0615caa0ad_9',
  'hyperparameters': None,
  'best_primary_metric': 0.9182094081942337,
  'status': 'Completed'}]

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

import pandas as pd
data = pd.concat([x,y], axis=1)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [12]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_cdf347c6-ef4b-4df3-8347-87ca894a4a4d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_cdf347c6-ef4b-4df3-8347-87ca894a4a4d',
 'target': 'amlcomp',
 'status': 'Completed',
 'startTimeUtc': '2023-02-01T05:46:25.221023Z',
 'endTimeUtc': '2023-02-01T06:24:32.820728Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'amlcomp',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-224190","workspace_name":"quick-starts-ws-224190","region":"southcentralus","compute_target":"amlcomp","spark_servic

In [15]:
# Retrieve and save your best automl model.

automl_best_run = remote_run.get_best_child()

automl_best_run.get_details()

{'runId': 'AutoML_cdf347c6-ef4b-4df3-8347-87ca894a4a4d_35',
 'target': 'amlcomp',
 'status': 'Completed',
 'startTimeUtc': '2023-02-01T06:23:05.420364Z',
 'endTimeUtc': '2023-02-01T06:24:29.697033Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'amlcomp\',\'subscription_id\':\'9e65f93e-bdd8-437b-b1e8-0647cd6098f7\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_cdf347c6-ef4b-4df3-8347-87ca894a4a4d_35","experiment_name":"udacity-project","workspace_name":"quick-starts-ws-224190","subscription_id":"9e65f93e-bdd8-

In [16]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_cdf347c6-ef4b-4df3-8347-87ca894a4a4d_35,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
# Cluster clean up

compute_target.delete()